<a href="https://colab.research.google.com/github/royaditya12/Computer-Vision-Projects-Self/blob/main/yolov8_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics roboflow wandb pytorch-grad-cam opencv-python-headless pillow matplotlib

In [ ]:
# Imports
import os
import wandb
import roboflow
from ultralytics import YOLO
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image

print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
WORKSPACE_NAME = "your-workspace"
PROJECT_NAME = "animals-object-detection-yolov8"
VERSION_NUMBER = 1

roboflow.login()

rf = roboflow.Roboflow()
project = rf.workspace(WORKSPACE_NAME).project(PROJECT_NAME)
dataset = project.version(VERSION_NUMBER).download("yolov8")

DATA_YAML_PATH = f"{dataset.location}/data.yaml"
print(f"Dataset downloaded to: {dataset.location}")
print(f"Data YAML path: {DATA_YAML_PATH}")

In [ ]:
wandb.login()

WANDB_PROJECT = "animals-object-detection-yolov8"

In [ ]:
# Initialize a W&B run
wandb.init(
    project=WANDB_PROJECT,
    name="baseline-run",
    config={
        "model": "yolov8n",
        "epochs": 50,
        "imgsz": 640,
        "batch": 16,
        "lr0": 0.01,
        "lrf": 0.01,
    }
)

# Load pretrained YOLOv8 model
model = YOLO("yolov8n.pt")

# Train the model
results = model.train(
    data=DATA_YAML_PATH,
    epochs=50,
    imgsz=640,
    batch=16,
    name="baseline_experiment",
    project="yolov8_training",
    patience=10,  # Early stopping patience
    save=True,
    plots=True,
)

wandb.finish()

In [ ]:
# Define sweep configuration
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'metrics/mAP50-95(B)',  # Metric to optimize
        'goal': 'maximize'
    },
    'parameters': {
        'epochs': {
            'value': 100
        },
        'imgsz': {
            'value': 640
        },
        'batch': {
            'values': [8, 16, 32]
        },
        'lr0': {
            'min': 0.0001,
            'max': 0.01,
            'distribution': 'log_uniform'
        },
        'lrf': {
            'min': 0.001,
            'max': 0.1,
            'distribution': 'log_uniform'
        },
        'momentum': {
            'min': 0.8,
            'max': 0.95
        },
        'weight_decay': {
            'min': 0.0001,
            'max': 0.001,
            'distribution': 'log_uniform'
        }
    }
}

# Initialize sweep
sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT, entity=WANDB_ENTITY)
print(f"Sweep ID: {sweep_id}")

In [ ]:
def train_with_sweep():
    run = wandb.init()
    config = wandb.config

    # Load model
    model = YOLO("yolov8n.pt")

    # Train with sweep parameters
    results = model.train(
        data=DATA_YAML_PATH,
        epochs=config.epochs,
        imgsz=config.imgsz,
        batch=config.batch,
        lr0=config.lr0,
        lrf=config.lrf,
        momentum=config.momentum,
        weight_decay=config.weight_decay,
        name=f"sweep_{run.id}",
        project="yolov8_sweep",
        patience=15,
        save=True,
        plots=False
    )

    wandb.finish()

In [ ]:
wandb.agent(sweep_id, function=train_with_sweep, count=10)

In [ ]:
# best hyperparameters obtained from W&B sweep dashboard
BEST_CONFIG = {
    "model": "yolov8n",
    "epochs": 150,
    "imgsz": 640,
    "batch": 16,
    "lr0": 0.005,
    "lrf": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
}

wandb.init(
    project=WANDB_PROJECT,
    entity=WANDB_ENTITY,
    name="final-best-model",
    config=BEST_CONFIG
)

model = YOLO("yolov8n.pt")

# Train with best hyperparameters
results = model.train(
    data=DATA_YAML_PATH,
    epochs=BEST_CONFIG["epochs"],
    imgsz=BEST_CONFIG["imgsz"],
    batch=BEST_CONFIG["batch"],
    lr0=BEST_CONFIG["lr0"],
    lrf=BEST_CONFIG["lrf"],
    momentum=BEST_CONFIG["momentum"],
    weight_decay=BEST_CONFIG["weight_decay"],
    hsv_h=BEST_CONFIG["hsv_h"],
    hsv_s=BEST_CONFIG["hsv_s"],
    hsv_v=BEST_CONFIG["hsv_v"],
    degrees=BEST_CONFIG["degrees"],
    flipud=BEST_CONFIG["flipud"],
    fliplr=BEST_CONFIG["fliplr"],
    mosaic=BEST_CONFIG["mosaic"],
    name="final_model",
    project="yolov8_final",
    patience=20,
    save=True,
    plots=True,
)

BEST_MODEL_PATH = "runs/detect/final_model/weights/best.pt"

wandb.finish()
print(f"\nFinal model saved at: {BEST_MODEL_PATH}")

In [ ]:
model = YOLO(BEST_MODEL_PATH)

metrics = model.val(data=DATA_YAML_PATH, split='test')

print("\n=== Test Set Evaluation ===")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP75: {metrics.box.map75:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

In [ ]:
TEST_IMAGE_PATH = "/content/test_images/image0102.jpg"

# Run inference
results = model.predict(
    source=TEST_IMAGE_PATH,
    conf=0.25,  # Confidence threshold
    iou=0.45,   # IoU threshold
    save=True,
    save_txt=True,
    save_conf=True,
    project="predictions",
    name="test_results"
)

for result in results:
    result.show()
    boxes = result.boxes
    print(f"\nDetected {len(boxes)} objects:")
    for box in boxes:
        class_id = int(box.cls[0])
        confidence = float(box.conf[0])
        class_name = model.names[class_id]
        print(f"  - {class_name}: {confidence:.2f}")